In [4]:
import pandas as pd
import numpy as np
import openpyxl
import os
import sys
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pyparsing import col
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm 
import re
import requests 

import re
import json

# 경고 메시지 무시 
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
geonggi_cafe = pd.read_csv('./add_to_filename.csv', encoding='utf-8')

In [6]:
geonggi_cafe.tail(3)

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input,split_list,input_search,stores_url,file_names
2713,베러댄레스트 카페,https://www.instagram.com/betterthan_rest,"음식점>카페,디저트",NaN,NaN,경기도 성남시 수정구 창곡동 537-3 지하 1층,경기도 성남시 수정구 위례서일로1길 19 지하 1층,323928,540950,경기도 하남시 위례동 애견카페,"카페,디저트",경기도 하남시 위례동 베러댄레스트 카페,https://m.map.naver.com/search2/search.naver?q...,베러댄레스트 카페.jpg
2714,카페일리터& 바 신구대점,NaN,"음식점>카페,디저트",NaN,NaN,경기도 성남시 중원구 금광동 3161 1층,경기도 성남시 중원구 금빛로61번길 14 1층,326357,538671,경기도 광주시 남한산성면 바(BAR),"카페,디저트",경기도 광주시 남한산성면 카페일리터& 바 신구대점,https://m.map.naver.com/search2/search.naver?q...,카페일리터& 바 신구대점.jpg
2715,이디야커피 성남상대원 시장 점,http://www.ediya.com/,"카페,디저트>카페",NaN,NaN,경기도 성남시 중원구 상대원동 2986-15,경기도 성남시 중원구 희망로 334,325706,537400,경기도 광주시 광남1동 시장,카페,경기도 광주시 광남1동 이디야커피 성남상대원 시장 점,https://m.map.naver.com/search2/search.naver?q...,이디야커피 성남상대원 시장 점.jpg


In [7]:
len(geonggi_cafe)

2716

In [8]:
# 0 ~ 839
geonggi_cafe[geonggi_cafe['address'].str.contains('용인시')]

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input,split_list,input_search,stores_url,file_names
0,Sucrami,https://instagram.com/sucrami,"음식점>카페,디저트",NaN,NaN,경기도 용인시 기흥구 영덕동 1001-2 1층,경기도 용인시 기흥구 흥덕2로65번길 5 1층,318043,519442,경기도 수원시 영통구 매탄3동 롤케익,"카페,디저트",경기도 수원시 영통구 매탄3동 Sucrami,https://m.map.naver.com/search2/search.naver?q...,Sucrami.jpg
1,커맨드커피,http://commandcoffee.co.kr,"음식점>카페,디저트",NaN,NaN,경기도 용인시 기흥구 영덕동 985-1,경기도 용인시 기흥구 흥덕2로87번길 18,317909,519717,경기도 수원시 영통구 매탄3동 인도음식,"카페,디저트",경기도 수원시 영통구 매탄3동 커맨드커피,https://m.map.naver.com/search2/search.naver?q...,커맨드커피.jpg
2,스타벅스 수원IC DT점,http://www.starbucks.co.kr/,"카페,디저트>카페",NaN,NaN,경기도 용인시 기흥구 영덕동 522-2 LG동광주유소,경기도 용인시 기흥구 중부대로 56,318142,518768,경기도 수원시 영통구 영통3동 다방,카페,경기도 수원시 영통구 영통3동 스타벅스 수원IC DT점,https://m.map.naver.com/search2/search.naver?q...,스타벅스 수원IC DT점.jpg
3,옐로버터,https://smartstore.naver.com/yolong_caron,"음식점>카페,디저트",NaN,NaN,경기도 용인시 기흥구 서천동 405-2,경기도 용인시 기흥구 서그내로62번길 49,317457,516036,경기도 수원시 영통구 영통3동 마카롱,"카페,디저트",경기도 수원시 영통구 영통3동 옐로버터,https://m.map.naver.com/search2/search.naver?q...,옐로버터.jpg
4,타니예르,NaN,"카페,디저트>베이커리",NaN,NaN,경기도 용인시 기흥구 서천동 730 TANIERE,경기도 용인시 기흥구 서천서로12번길 4-2 TANIERE,318008,515823,경기도 수원시 영통구 영통3동 베이커리,베이커리,경기도 수원시 영통구 영통3동 타니예르,https://m.map.naver.com/search2/search.naver?q...,타니예르.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,리나커피,http://instargram.com/lina.coffii,"음식점>카페,디저트",NaN,NaN,경기도 용인시 수지구 상현동 1220 제지1층 제알동 B126호,경기도 용인시 수지구 광교중앙로 338 제지1층 제알동 B126호,317862,522484,경기도 용인시 수지구 상현1동 애견카페,"카페,디저트",경기도 용인시 수지구 상현1동 리나커피,https://m.map.naver.com/search2/search.naver?q...,리나커피.jpg
836,젤라커피,NaN,"음식점>카페,디저트",NaN,NaN,경기도 용인시 수지구 상현동 1158-7 1층,경기도 용인시 수지구 광교호수로378번길 16 1층,318684,522109,경기도 용인시 수지구 상현1동 애견카페,"카페,디저트",경기도 용인시 수지구 상현1동 젤라커피,https://m.map.naver.com/search2/search.naver?q...,젤라커피.jpg
837,카페 888,https://www.instagram.com/yicafe888,"음식점>카페,디저트",NaN,NaN,경기도 용인시 처인구 이동읍 묘봉리 866-3,경기도 용인시 처인구 이동읍 경기동로 888,329790,502855,경기도 안성시 양성면 북카페,"카페,디저트",경기도 안성시 양성면 카페 888,https://m.map.naver.com/search2/search.naver?q...,카페 888.jpg
838,리엔호이스,NaN,"카페,디저트>카페",NaN,NaN,경기도 용인시 처인구 이동읍 어비리 962,경기도 용인시 처인구 이동읍 어진로 798,328520,500904,경기도 안성시 양성면 수상스키,카페,경기도 안성시 양성면 리엔호이스,https://m.map.naver.com/search2/search.naver?q...,리엔호이스.jpg


In [9]:
# 840 ~ 1826
geonggi_cafe[geonggi_cafe['address'].str.contains('수원시')]

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input,split_list,input_search,stores_url,file_names
840,아주좋은우리동네커피집,NaN,"카페,디저트>카페",NaN,NaN,경기도 수원시 장안구 조원동 101-4,경기도 수원시 장안구 송원로 125,312659,523048,경기도 수원시 장안구 파장동 과일쥬스,카페,경기도 수원시 장안구 파장동 아주좋은우리동네커피집,https://m.map.naver.com/search2/search.naver?q...,아주좋은우리동네커피집.jpg
841,뜨레모아 과자 점,NaN,"카페,디저트>베이커리",NaN,NaN,경기도 수원시 장안구 파장동 579-36,경기도 수원시 장안구 파장로68번길 34,310902,523309,경기도 수원시 장안구 파장동 과자,베이커리,경기도 수원시 장안구 파장동 뜨레모아 과자 점,https://m.map.naver.com/search2/search.naver?q...,뜨레모아 과자 점.jpg
842,고운 맘씨,https://instagram.com/cafe_a_kind_mom?igshid=k...,"음식점>카페,디저트",NaN,NaN,경기도 수원시 장안구 파장동 267-3 1층 고운 맘씨,경기도 수원시 장안구 경수대로1078번길 11 1층 고운 맘씨,311093,523588,경기도 수원시 장안구 파장동 과자,"카페,디저트",경기도 수원시 장안구 파장동 고운 맘씨,https://m.map.naver.com/search2/search.naver?q...,고운 맘씨.jpg
843,스타벅스 북수원IC DT점,http://www.starbucks.co.kr/,"카페,디저트>카페",NaN,NaN,"경기도 수원시 장안구 파장동 469-4 469-47, 469-48, 469-49번지...",경기도 수원시 장안구 경수대로 1038,311428,523344,경기도 수원시 장안구 파장동 다방,카페,경기도 수원시 장안구 파장동 스타벅스 북수원IC DT점,https://m.map.naver.com/search2/search.naver?q...,스타벅스 북수원IC DT점.jpg
844,뚱보네꽈배기,NaN,"카페,디저트>베이커리",NaN,NaN,경기도 수원시 장안구 파장동 579-3 뚱보네꽈배기,경기도 수원시 장안구 경수대로1081번길 41 뚱보네꽈배기,310919,523336,경기도 수원시 장안구 파장동 도넛,베이커리,경기도 수원시 장안구 파장동 뚱보네꽈배기,https://m.map.naver.com/search2/search.naver?q...,뚱보네꽈배기.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,굼굼당,https://www.instagram.com/goomgoom_dang,"음식점>카페,디저트",NaN,NaN,경기도 수원시 권선구 호매실동 1344-4 1층 굼굼당,경기도 수원시 권선구 금곡로140번길 95-9 1층 굼굼당,306565,519152,경기도 화성시 매송면 애견카페,"카페,디저트",경기도 화성시 매송면 굼굼당,https://m.map.naver.com/search2/search.naver?q...,굼굼당.jpg
1823,댕들 신동 카페 거리점,http://instagram.com/dangdle_official,"음식점>카페,디저트",NaN,NaN,경기도 수원시 영통구 신동 990 1F,경기도 수원시 영통구 권선로882번길 108-12 1F,315106,516475,경기도 화성시 반월동 애견카페,"카페,디저트",경기도 화성시 반월동 댕들 신동 카페 거리점,https://m.map.naver.com/search2/search.naver?q...,댕들 신동 카페 거리점.jpg
1824,카페플러스셀,NaN,"카페,디저트>베이커리",NaN,NaN,경기도 수원시 권선구 오목천동 775 1층 우측,경기도 수원시 권선구 서수원로53번길 25-36 1층 우측,309504,515000,경기도 화성시 기배동 수영장,베이커리,경기도 화성시 기배동 카페플러스셀,https://m.map.naver.com/search2/search.naver?q...,카페플러스셀.jpg
1825,카페 여름가을,https://instagram.com/cafe_yeoreum._.gaeul,"음식점>카페,디저트",NaN,NaN,경기도 수원시 권선구 오목천동 1180 309동 1층 109호,경기도 수원시 권선구 서수원로 99 309동 1층 109호,309254,515411,경기도 화성시 기배동 애견카페,"카페,디저트",경기도 화성시 기배동 카페 여름가을,https://m.map.naver.com/search2/search.naver?q...,카페 여름가을.jpg


In [10]:
# 1827 ~ 
geonggi_cafe[geonggi_cafe['address'].str.contains('성남시')]

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input,split_list,input_search,stores_url,file_names
1827,커피나무,NaN,"카페,디저트>카페",NaN,NaN,경기도 성남시 수정구 신흥동 5536 신흥상가 1층,경기도 성남시 수정구 수정로 160 신흥상가 1층,323909,538180,경기도 성남시 수정구 신흥1동 과일쥬스,카페,경기도 성남시 수정구 신흥1동 커피나무,https://m.map.naver.com/search2/search.naver?q...,커피나무.jpg
1828,카페베네 수진역점,https://www.instagram.com/cafebene_sujin/?igsh...,"카페,디저트>카페",NaN,NaN,경기도 성남시 중원구 성남동 2126,경기도 성남시 중원구 산성대로 208 1층 카페베네,324234,537613,경기도 성남시 수정구 신흥1동 과일쥬스,카페,경기도 성남시 수정구 신흥1동 카페베네 수진역점,https://m.map.naver.com/search2/search.naver?q...,카페베네 수진역점.jpg
1829,무인카페428,NaN,"카페,디저트>카페",NaN,NaN,경기도 성남시 수정구 신흥동 4124 롯데시네마 4층 428호,경기도 성남시 수정구 산성대로 267 롯데시네마 4층 428호,324668,538039,경기도 성남시 수정구 신흥1동 과자,카페,경기도 성남시 수정구 신흥1동 무인카페428,https://m.map.naver.com/search2/search.naver?q...,무인카페428.jpg
1830,스타벅스 신흥역점,http://www.starbucks.co.kr/,"카페,디저트>카페",NaN,NaN,"경기도 성남시 수정구 신흥동 4124 신흥동롯데시네마타워 120호, 120-1호, ...",경기도 성남시 수정구 산성대로 267,324684,538006,경기도 성남시 수정구 신흥1동 다방,카페,경기도 성남시 수정구 신흥1동 스타벅스 신흥역점,https://m.map.naver.com/search2/search.naver?q...,스타벅스 신흥역점.jpg
1831,캔 다방 성남신흥점,NaN,"음식점>카페,디저트",NaN,NaN,경기도 성남시 수정구 신흥동 5192 1층,경기도 성남시 수정구 수정남로 134 1층,324298,538239,경기도 성남시 수정구 신흥1동 다방,"카페,디저트",경기도 성남시 수정구 신흥1동 캔 다방 성남신흥점,https://m.map.naver.com/search2/search.naver?q...,캔 다방 성남신흥점.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,그린그라스,https://www.instagram.com/greengrass__official/,"카페,디저트>카페",NaN,NaN,경기도 성남시 분당구 석운동 52-5 카페 그린그라스,경기도 성남시 분당구 석운로202번길 1 카페 그린그라스,315691,531402,경기도 의왕시 청계동 애견카페,카페,경기도 의왕시 청계동 그린그라스,https://m.map.naver.com/search2/search.naver?q...,그린그라스.jpg
2712,소녀다방,http://www.instagram.com/cafe._.dabang,"음식점>카페,디저트",NaN,NaN,경기도 성남시 수정구 창곡동 548-5 1층 소녀다방,경기도 성남시 수정구 위례서일로3길 14-5 1층 소녀다방,324091,540813,경기도 하남시 위례동 애견카페,"카페,디저트",경기도 하남시 위례동 소녀다방,https://m.map.naver.com/search2/search.naver?q...,소녀다방.jpg
2713,베러댄레스트 카페,https://www.instagram.com/betterthan_rest,"음식점>카페,디저트",NaN,NaN,경기도 성남시 수정구 창곡동 537-3 지하 1층,경기도 성남시 수정구 위례서일로1길 19 지하 1층,323928,540950,경기도 하남시 위례동 애견카페,"카페,디저트",경기도 하남시 위례동 베러댄레스트 카페,https://m.map.naver.com/search2/search.naver?q...,베러댄레스트 카페.jpg
2714,카페일리터& 바 신구대점,NaN,"음식점>카페,디저트",NaN,NaN,경기도 성남시 중원구 금광동 3161 1층,경기도 성남시 중원구 금빛로61번길 14 1층,326357,538671,경기도 광주시 남한산성면 바(BAR),"카페,디저트",경기도 광주시 남한산성면 카페일리터& 바 신구대점,https://m.map.naver.com/search2/search.naver?q...,카페일리터& 바 신구대점.jpg


In [11]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import time
import urllib

driver = webdriver.Chrome('chromedriver.exe')  # 드라이버 경로
# driver = webdriver.Chrome('chromedriver',chrome_options=options) # 크롬창 숨기기

In [12]:
# 장소 이미지 담을 디렉토리 생성
def createDir(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            print("Create : Success to create the directory.")
    except OSError:
        print("Error: Failed to create the directory.")

def delDir(directory):
    try:
        if os.path.exists(directory):
            os.rmdir(directory)
            print("Delete : Success to delete the directory.")
    except OSError:
        print("Not Empty : IMG Exists.")
        print("That's Okay!!! :)")
        
# 해당 xpath 존재여부 체크 함수
def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH, (xpath))
        return xpath
    except NoSuchElementException:
        return False
    return xpath

# 대표 이미지 존재 여부 체크 함수
def check_exists_by_img(xpath):
    try:
        driver.find_element(By.XPATH, (xpath))
    except NoSuchElementException:
        return False
    return True

In [13]:
def scrapping_img(place_url, place_name):
    driver = webdriver.Chrome('chromedriver.exe')  # 드라이버 경로
    
    url = place_url
    driver.get(url) # get : 사람이 url창에 입력하고 엔터치는거랑 똑같은 기능을 함
    time.sleep(3)
    print('가게명 : ', place_name)
    print('----- 1. url 이동 -----')
    
    # 검색어 최상단 스토어 클릭
    print('----- 2. 대표 사진 여부 -> xpath 위치 결정 -----')
    if len(driver.find_elements(By.XPATH, '//*[@id="ct"]/div[2]/ul/li/div[1]')) >= 1:
        if check_exists_by_img('//*[@id="ct"]/div[2]/ul/li/div[1]/a[1]') == True:
            button = driver.find_element(By.XPATH,('//*[@id="ct"]/div[2]/ul/li/div[1]/a[2]/div/strong'))
            button.click()
            time.sleep(2)
        else:
            button = driver.find_element(By.XPATH,('//*[@id="ct"]/div[2]/ul/li/div[1]/a/div/strong'))
            button.click()
            time.sleep(2)
    else: 
        print('가게 존재 X')
            
    time.sleep(2)
    # 현재 url의 앞쪽을 불러옴
    curr_url = driver.current_url
    curr_url = curr_url[:-4]
    
    review_url = curr_url + 'photo?filterType=내부'
    review_url
    
    # 사진 내부 url 로 이동
    print('----- 3. 사진 내부 url 로 이동 -----')
    driver.get(review_url)
    time.sleep(3)
    
    # 스크롤을 끝까지 내림 (END 입력!)
    # driver.find_element(By.TAG_NAME,'body').send_keys(Keys.PAGE_DOWN)
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(5)
    driver.find_element(By.TAG_NAME,'body').send_keys(Keys.END)
    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    time.sleep(3)
    try:
        print('----- 4. 이미지 생성 들어가기 -----')
        inner_img_lists = soup.select('#app-root > div > div > div > div:nth-child(7) > div:nth-child(4) > div.place_section_content > div.Nd2nM > div > div')
            
        dir = 'inner_img' + '/' + place_name + '/'
        
        print('----- 5. 디렉토리 생성하기 -----')
        createDir(dir)
        
        print('이미지 개수 : ', len(inner_img_lists))
        print('----- 6. 이미지 생성하기 -----')
        for img_num in range(1, len(inner_img_lists)+1):
            inner_img = driver.find_element(By.XPATH,('//*[@id="app-root"]/div/div/div/div[7]/div[4]/div/div/div/div'+'['+str(img_num)+']'+'/a/img')).get_attribute("src") #  "src" 속성을 받아옴
            urllib.request.urlretrieve(inner_img, dir + place_name + '_' + str(img_num) + ".jpg")
            print(img_num)
        
        # 비어있는 디렉토리 삭제
        print('----- 추가 작업 : 비어있는 디렉토리 삭제하기 (내부 이미지 X의 경우) -----')
        delDir(dir)
        print('Complete!!!!!!!!!!!!!!')
        print('')
        
    except NoSuchElementException:
        time.sleep(2)
        print('장소 존재 X')
        print('Complete!!!!!!!!!!!!!!')
        print('')

In [14]:
# 샘플 3개
cnt = 1
for place in range(0, 3):
    print('------------[', cnt, ']------------')
    scrapping_img(geonggi_cafe['stores_url'][place], geonggi_cafe['title'][place])
    cnt += 1

------------[ 1 ]------------
가게명 :  Sucrami
----- 1. url 이동 -----
----- 2. 대표 사진 여부 -> xpath 위치 결정 -----
----- 3. 사진 내부 url 로 이동 -----
----- 4. 이미지 생성 들어가기 -----
----- 5. 디렉토리 생성하기 -----
Create : Success to create the directory.
이미지 개수 :  13
----- 6. 이미지 생성하기 -----
1
2
3
4
5
6
7
8
9
10
11
12
13
----- 추가 작업 : 비어있는 디렉토리 삭제하기 (내부 이미지 X의 경우) -----
Not Empty : IMG Exists.
That's Okay!!! :)
Complete!!!!!!!!!!!!!!

------------[ 2 ]------------
가게명 :  커맨드커피
----- 1. url 이동 -----
----- 2. 대표 사진 여부 -> xpath 위치 결정 -----
----- 3. 사진 내부 url 로 이동 -----
----- 4. 이미지 생성 들어가기 -----
----- 5. 디렉토리 생성하기 -----
Create : Success to create the directory.
이미지 개수 :  10
----- 6. 이미지 생성하기 -----
1
2
3
4
5
6
7
8
9
10
----- 추가 작업 : 비어있는 디렉토리 삭제하기 (내부 이미지 X의 경우) -----
Not Empty : IMG Exists.
That's Okay!!! :)
Complete!!!!!!!!!!!!!!

------------[ 3 ]------------
가게명 :  스타벅스 수원IC DT점
----- 1. url 이동 -----
----- 2. 대표 사진 여부 -> xpath 위치 결정 -----
----- 3. 사진 내부 url 로 이동 -----
----- 4. 이미지 생성 들어가기 -----
----- 5. 디렉토리 생성